# Depth to Colormap Conversion Instructions

**Input:** .zip file of .csv depth files

**Output:** .zip file of cleaned .csv depth files, .png colorized depth files, and .png figures

> 1. Take all of your selected .csv depth files that you want to colorize and compress them into a .zip file with the name of **depth_data**, and
drag it into the files folder in the left sidebar.
>
> 2. Run sections **Imports**, **Upload Depth Data (CSV File)**, **Clean Data Functions**, **Depth to Cmap Conversion**, **Create Figure**, and **Remove All Files**.
>
> 3. Run section **Run**. Your depth data will begin being cleaned and colorized. Please be patient as it may take a moment depending on your input size.
>
> 4. Download the now available .zip file **colorized_depth_data.zip** from the file folder in the sidebar. If it is not there, click the Refresh Files button in the left sidebar. Once the download is finished, extract all of its contents to view the colorized depth data.
>
> 5. To convert another .zip file of depth data to colormaps, click **Runtime > Factory reset runtime**. Then, run **Imports** (or any code block EXCEPT for Run) to allow file uploads. After that, repeat steps 1-4.


> OPTIONAL: If you would like to experiment with different types of colormaps, go to the code block under **Depth to Cmap Conversion > Colormap Type**


# Imports



In [ ]:
import csv
import cv2
import os
import numpy as np
import pandas as pd
from PIL import Image
import PIL.Image as pil
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
from scipy import misc
from google.colab import files
import zipfile as zf
import shutil

# Upload Depth Data (CSV File)

In [ ]:
def setup():
  os.makedirs("all_data")

  # Make necessary directories
  dirs = ["all_data/figures", "all_data/colorized"]
  for dir in dirs: os.makedirs(dir)

  # Unzip and extract depth data from zip file
  files = zf.ZipFile("depth_data.zip", 'r')
  files.extractall('all_data/depth_data')
  files.close()

  # Get list of depth data file names
  file_list = os.listdir('all_data/depth_data')

  return file_list

# Clean Data Functions

## Get Max and Min Values of CSV Function

In [ ]:
def get_range(df):
  max = []
  min = []
  
  # Find and saves max and min of each column
  for column in df:
    max.append(df[column].max())
    min.append(df[column].min())

  return np.max(max), np.min(min)

## Replace All False Readings

In [ ]:
def remove_false_readings(csv_filename):
  # Load CSV into pandas dataframe
  df=pd.read_csv("all_data/depth_data/" + csv_filename)

  # # Check current max and min depth values
  # max_depth, min_depth = get_range(df)
  # print("Current Max:", max_depth, "\nCurrent Min:", min_depth)

  # # Keep track of false depth readings
  # badReadingCount = 0

  # Iterate over all cells of CSV
  for column in df:
    row_index = 0
    for value in df[column]:
      # Replace all false readings with 0
      if value > 6:
        # badReadingCount += 1
        df[column][row_index] = 0
      row_index += 1

  # Save cleaned data to new CSV
  df.to_csv("all_data/depth_data/" + csv_filename, index=False, header=False)

  # Get new max and min values
  max_depth, min_depth = get_range(df)

  # # Check new max and min values
  # print("New Max:", max_depth, "\nNew Min:", min_depth)

  # # Check total number of false readings
  # print("Bad reading count:", badReadingCount)

  return max_depth, min_depth

# Depth to Cmap Conversion

## Colormap Type

In [ ]:
cmap_type = 'rainbow'

# For a list of different colormap types, visit https://www.kite.com/python/docs/matplotlib.pyplot.colormaps

## Create Colormap Using .imshow()

In [ ]:
# Create color map using Matplotlib
def create_cmap(csv_filename):
  with open("all_data/depth_data/" + csv_filename, encoding='UTF-8-sig') as csvfile:
    depth_data = np.loadtxt(csvfile, delimiter=",", skiprows=0)  #Get the data in the file
    fig = plt.imshow(depth_data, cmap=cmap_type)
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)

    plt.savefig("all_data/colorized/" + csv_filename.replace(".csv", "_colorized.png"), bbox_inches='tight')
    plt.close()

  return depth_data

# Create Figure

In [ ]:
def create_figure(csv_filename, max_depth, min_depth, depth_data):
  normalizer = mpl.colors.Normalize(vmin=min_depth, vmax=max_depth)
  mapper = cm.ScalarMappable(norm=normalizer, cmap=cmap_type)

  fig, ax = plt.subplots()
  fig.colorbar(mapper)
  plt.imshow(depth_data, cmap=cmap_type)
  fig_name = "all_data/figures/" + csv_filename.replace(".csv", "") + "_figure.png"
  plt.savefig(fig_name, bbox_inches='tight')
  plt.close()

  return None

# Remove All Files

In [ ]:
def remove_all_files(del_zip=False):
  ignore = ['.config', '.ipynb_checkpoints']
  for file in os.listdir():
    if file in ignore:
      continue
    elif file == 'colorized_depth_data.zip':
      os.remove(file)
      continue
    elif file == 'depth_data.zip' and del_zip == False:
      continue
    elif file == 'depth_data.zip' and del_zip == True:
      os.remove(file)
      continue
    else:
      for dir in os.listdir(file):
        for f in os.listdir(file + "/" + dir):
          os.remove(file + "/" + dir + "/" + f)
        os.rmdir(file + "/" + dir)
      os.rmdir(file)
  
  return None

# Run

In [ ]:
# remove_all_files()
file_list = setup()

for file in file_list:
  if file == ".ipynb_checkpoints":
    continue
  max_depth, min_depth = remove_false_readings(file)
  depth_data = create_cmap(file)
  create_figure(file, max_depth, min_depth, depth_data)

shutil.make_archive("colorized_depth_data", 'zip', "all_data")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'/content/colorized_depth_data.zip'